# Get MIRI programs info #

**Goal:** Create a csv file with relevant information on all observed MIRI programs or a part and download data from MAST

**Author:** Boris Trahin, Staff Scientist II, MIRI team

**Last updated:** June 29, 2024

***
### Import useful packages

In [ ]:
from astroquery.mast.missions import MastMissions
from astroquery.mast import Observations
from astropy.table import unique, vstack, Table
from datetime import date
import pandas as pd

***
### Get entire catalog of MIRI observations

<div class="alert alert-block alert-info">
Either get all the data from MAST or directly read the csv file downloaded from MAST
</div>

In [ ]:
mast = False # False = get info directly from the csv file downloaded from MAST
csv_file = './JWST_2024-08-06T08_53_31-04_00.csv' # csv file
save = True # save the catalogs into csv file

good_columns = ['obs_id', 'fileSetName', 'program', 'proposal_cycle', 'date_obs', 'visitgrp',
       'observtn', 'bkgdtarg', 'visit',
       'targtype', 'exp_type', 'tsovisit', 'expripar', 'detector',
       'opticalElements', 'filter', 'subarray', 'pps_aper', 'apername',
       'miri_channel', 'miri_band', 'miri_coronmsk', 'targ_ra', 'targ_dec',
       'nresets', 'nframes', 'ngroups', 'nints', 'readpat', 'numdthpt',
       'patttype', 'miri_dithopfr', 'miri_numdsets',
       'tframe', 'tgroup', 'effinttm','duration', 'eng_qual',
       'hga_move', 'lamp', 'pcs_mode', 'miri_cccstate', 'exp_only', 'visitsta', 'template',
       'targdesc', 'scicat', 'targcat', 'targprop', 'targname', 'obslabel', 'title',
       'pi_name', 'category', 'access']

if mast:
    missions = MastMissions(mission='jwst')
    results = missions.query_criteria(select_cols=good_columns,
    instrume='MIRI',
    limit=-1        
        )
    df = results.to_pandas()
    update = date.today()
else:
    df = pd.read_csv(csv_file, low_memory=False)
    bad_columns = [x for x in df.columns if x not in good_columns]
    df = df.drop(bad_columns, axis='columns')
    update = csv_file.split('_')[1][:10]

<div class="alert alert-block alert-info">
Some columns are badly reference (e.g. MRS detector), so organize it.
One can uncomment the line df2 = df2.drop_duplicates(subset=['obs_id', 'exp_type']) to remove duplicates based on obs_id and exp_type. This will keep only 1 dither position per exposure.
</div>

In [ ]:
df2 = df.reindex(good_columns, axis=1)

# Some columns are badly referenced
df2['detector'] = df2['detector'].fillna('')
df2['detector'] = df2['detector'].astype(str)
df2.loc[(df2['opticalElements'].str.contains('CH12')) & (df2['opticalElements'].str.contains('CH34')) & (df2['detector'].str.contains('MIRIFU')), 'detector'] = 'MIRIFUSHORT, MIRIFULONG'
df2.loc[(df2['opticalElements'].str.contains('CH12')) & (df2['opticalElements'].str.contains('CH34')) & (df2['opticalElements'].str.contains('F')) & (df2['detector'].str.contains('MIRIFU')), 'detector']  = 'MIRIFUSHORT, MIRIFULONG, MIRIMAGE'

for index, row in df2.iterrows():
    if ('CH12' in str(df2['opticalElements'][index])) & ('CH34' in str(df2['opticalElements'][index])) & ('F' in str(df2['opticalElements'][index])):
        df2.loc[index,'filter'] = df2.loc[index,'opticalElements'].split(',')[-1].replace(',','-')
        
df2.loc[(df2['exp_type'].astype(str).str.contains('MIR_DARKMRS', na=False)), 'detector'] = 'MIRIFUSHORT, MIRIFULONG'

df2['miri_channel'] = df2['miri_channel'].fillna('')
df2['miri_channel'] = df2['miri_channel'].astype(str)
df2.loc[(df2['opticalElements'].str.contains('CH12')) & (df2['opticalElements'].str.contains('CH34')) & (df2['detector'].str.contains('MIRIFU')), 'miri_channel'] = '12, 34'
df2.loc[(df2['opticalElements'].str.contains('OPAQUE')) & (df2['detector'].str.contains('MIRIFU')), 'miri_channel'] = '12, 34'

df2['readpat'] = df2['readpat'].astype(str)
df2.loc[(df2['tgroup']<3) & (df2['nresets']==1), 'readpat'] = 'FASTR1'
df2.loc[(df2['tgroup']<3) & (df2['nresets']==0), 'readpat'] = 'FAST'
df2.loc[(df2['tgroup']>3) & (df2['nresets']==1), 'readpat'] = 'SLOWR1'
df2.loc[(df2['tgroup']>3) & (df2['nresets']==0), 'readpat'] = 'SLOW'

df2 = df2.drop(df2[df2['fileSetName'].str.contains('-')].index)

df2 = df2.sort_values(['program', 'observtn'],axis=0, ascending=False) # latest program first

# df2 = df2.drop_duplicates(subset=['obs_id', 'exp_type']) # Uncomment this line to remove duplicates based on obs_id and exp_type. This will keep only 1 dither position per exposure.

if save:
    with open(f'./JWST_MIRI_programs_info_{update}.csv', 'w') as outfile:
        # outfile.write(f'{date.today()}')
        # outfile.write(f'\n')
        df2.to_csv(outfile, index=False) # save to a csv file and remove first index column

***
### Search in the full catalog and create a sub-catalog

<div class="alert alert-block alert-info">
Change conditions in df_sort to create a sub-catalog
</div>

In [ ]:
# Search with conditions
# condition = (df2['exp_type'].str.contains('IMAGE')) & (df2['ngroups']>100) & (df2['subarray'].str.contains('64'))
condition = (df2['detector'].str.contains('MIRIMAGE') & df2['subarray'].str.contains('FULL'))
# condition = (df2['exp_type'].str.contains('DARK')) & (df2['ngroups']>40) & (df2['subarray']=='FULL')

df_sort = df2[condition]

if save:
    outfile = f'./Sorted_Catalog_MIRI.csv'
    df_sort.to_csv(outfile, index=False) # save to a csv file and remove first index column

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# df_sort

***
### Download data from MAST

<div class="alert alert-block alert-info">
Download the data
</div>

In [ ]:
# Download data
search_mast_data = False
download_mast_data = False
download_dir = '/Users/btrahin/Data/'

obs_ids = ['jw02114004001_04101_00002']
extension = 'RATEINTS'

In [ ]:
if search_mast_data:
    files = []
    for i in obs_ids:
        print(f'{obs_ids.index(i)+1}/{len(obs_ids)}. Getting observation {i} products...')
        pid = i[2:7]
        visitid = i[7:10]
        obs = Observations.query_criteria(
            obs_collection = 'JWST',
            instrument_name = ['MIRI*'],
            obs_id = f'jw{pid}*{visitid}*',
            )
        t = Observations.get_product_list(obs)
        mask = (t['productType'] =='SCIENCE') & (t['productSubGroupDescription'] == extension)
        t = t[mask]
        for x in range(len(t)):
            if (i in t['obs_id'][x]) and ('mirifulong' in t['obs_id'][x]):
                files.append(t[x])
    allfiles = unique(vstack(files), keys='productFilename')
    products = Observations.filter_products(allfiles, extension='fits')

In [ ]:
if download_mast_data:
    personal_token = ''
    Observations.login(token=personal_token)
    Observations.download_products(
        products,
        curl_flag=True, # if True, a curl script will be downloaded that can be used to download the data files at a later time. Then run sh *file* in terminal.
        download_dir=download_dir
        )